# Books Database Analysis on SQL 

## Introduction

A startup that develops app for books lovers give us a database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product. 

We will conduct the analysis with SQL queries and provide conclusions and recommendations at the end of the report.

In [6]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

We will print each dataset's first five rows to get a glance of each data.

In [7]:
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = ''' SELECT *
            FROM ratings
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = ''' SELECT *
            FROM reviews
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query = ''' SELECT *
            FROM publishers
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Conclusion
Most of the dataset looks good, except some of the columns may need to be converted into different data type.

## Tasks

The following are the tasks that we will complete:
- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating: look only at books with at least 50 ratings.
- Find the average number of text reviews among users who rated more than 50 books.

### Books released after January 1, 2020

In [7]:
query = ''' SELECT COUNT(title)
            FROM books
            WHERE CAST(publication_date AS date) > '2000-01-01'
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


In [8]:
query = ''' SELECT COUNT(title)
            FROM books
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,1000


From this, we can see that there are 819 books released after January 1, 2000. It comprises more than 80% of all books in the database. Therefore we can conclude that the database has relatively new books and less classic books, however it is possible that among these books there are classic ones that were re-published after January 1, 2000.

### Average rating and total reviews of each book

In [13]:
query = ''' SELECT 
                v.book_id AS book_id,
                AVG(t.rating) AS avg_rating,
                COUNT(v.review_id) AS total_reviews
            FROM 
                reviews AS v
                INNER JOIN ratings AS t ON v.book_id = t.book_id
            GROUP BY 1
            ORDER BY 3 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,avg_rating,total_reviews
0,948,3.662500,1120
1,750,4.125000,528
2,673,3.825581,516
3,302,4.414634,492
4,299,4.287500,480


In [10]:
query = ''' SELECT COUNT (DISTINCT book_id)
            FROM reviews
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,994


In the table above, we can see that there are only 994 books in total that has reviews (around 0.06% missing reviews), the rest only have ratings. We should check these 6 books.

In [14]:
query = ''' SELECT 
                b.book_id,
                b.title,
                AVG(t.rating) AS avg_rating
            FROM
                books AS b
                LEFT JOIN reviews AS v ON b.book_id = v.book_id
                LEFT JOIN ratings AS t ON b.book_id = t.book_id
            WHERE v.review_id IS NULL
            GROUP BY 1
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating
0,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667
1,191,Disney's Beauty and the Beast (A Little Golden...,4.000000
2,221,Essential Tales and Poems,4.000000
3,387,Leonardo's Notebooks,4.000000
4,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000
5,808,The Natural Way to Draw,3.000000


The books in the table above do not have any text reviews but it has its average rating. Even one of the books have a rating of 5, The Cat in the Hat and Other Dr. Seuss Favorites.

In [17]:
query = ''' SELECT 
                b.book_id AS book_id,
                b.title AS title,
                AVG(t.rating) AS avg_rating,
                COUNT(DISTINCT v.review_id) AS total_reviews
            FROM 
                books AS b
                LEFT JOIN ratings AS t ON b.book_id = t.book_id
                LEFT JOIN reviews AS v ON b.book_id = v.book_id
            GROUP BY 1, 2
            ORDER BY 3 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating,total_reviews
0,55,A Woman of Substance (Emma Harte Saga #1),5.0,2
1,57,Act of Treason (Mitch Rapp #9),5.0,2
2,17,A Dirty Job (Grim Reaper #1),5.0,4
3,20,A Fistful of Charms (The Hollows #4),5.0,2
4,62,Alas Babylon,5.0,2


In [19]:
query = ''' SELECT 
                b.book_id AS book_id,
                b.title AS title,
                AVG(t.rating) AS avg_rating,
                COUNT(DISTINCT v.review_id) AS total_reviews
            FROM 
                books AS b
                LEFT JOIN ratings AS t ON b.book_id = t.book_id
                LEFT JOIN reviews AS v ON b.book_id = v.book_id
            GROUP BY 1, 2
            ORDER BY 4 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,avg_rating,total_reviews
0,948,Twilight (Twilight #1),3.662500,7
1,497,Outlander (Outlander #1),4.125000,6
2,207,Eat Pray Love,3.395833,6
3,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
4,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6


From the tables above, we can see that when we sort the table by highest rating, it usually comes back with books with very little amount of total reviews. But when we sort with books with most number of distinct reviews, the rating is more varied, ranging from 3.0-4.5.

This means that the more popular a book is, the more varied the reviews will be as people have different preference and perspective on books, a good book for one person may not be good for another person. A good indicator of a great book is when they score high on rating and have a lot of reviews. In this case, Harry Potter and the Prisoner of Azkaban, a popular book indeed!

### Publishers with most books

In [12]:
query = ''' SELECT 
                b.publisher_id,
                p.publisher,
                COUNT(b.book_id) AS total_books
            FROM 
                books AS b
                INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id 
            WHERE b.num_pages > 50
            GROUP BY 1, 2
            ORDER BY 3 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher,total_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19


In [20]:
query = ''' SELECT
                b.title AS title,
                AVG(t.rating) AS avg_rating
            FROM 
                books AS b
                INNER JOIN ratings AS t ON b.book_id = t.book_id
            WHERE publisher_id = 212
            GROUP BY 1
            ORDER BY 2 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,title,avg_rating
0,A Princess of Mars (Barsoom #1),4.5
1,The Body,4.5
2,On Beauty,4.5
3,Lost in a Good Book (Thursday Next #2),4.5
4,Icy Sparks,4.5


Based on the database, the publisher with most books published is Penguin Books with 42 books. Their most highly rated books are A Princess of Mars, The Body and On Beauty.

### Author with highest average rating

In [22]:
query = ''' SELECT
                b.title AS title,
                a.author AS author,
                AVG(t.rating) AS avg_rating
            FROM 
                books AS b
                INNER JOIN authors AS a ON b.author_id = a.author_id
                INNER JOIN ratings AS t ON b.book_id = t.book_id
            GROUP BY 1, 2
            HAVING COUNT(t.rating) >= 50
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,title,author,avg_rating
0,Of Mice and Men,John Steinbeck,3.622951
1,The Alchemist,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
2,Animal Farm,George Orwell/Boris Grabnar/Peter Škerl,3.729730
3,Romeo and Juliet,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
4,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.186667


In [26]:
query = ''' SELECT
                COUNT(*)
            FROM (SELECT b.title AS title,
                a.author AS author,
                AVG(t.rating) AS avg_rating
            FROM 
                books AS b
                INNER JOIN authors AS a ON b.author_id = a.author_id
                INNER JOIN ratings AS t ON b.book_id = t.book_id
            GROUP BY 1, 2
            HAVING COUNT(t.rating) >= 50) AS total
        '''

pd.io.sql.read_sql(query, con = engine)

,count
0,20


There are only 20 authors that have more than 50 ratings. From this list we would count the average rating grouped by the author's name.

In [34]:
query = ''' SELECT
                ar.author_id,
                ar.author,
                AVG(ar.avg_rating) AS avg_rating
            FROM (SELECT
                    b.book_id AS book_id,
                    a.author_id AS author_id,
                    b.title AS title,
                    a.author AS author,
                    AVG(t.rating) AS avg_rating
                FROM 
                    books AS b
                    INNER JOIN authors AS a ON b.author_id = a.author_id
                INNER JOIN ratings AS t ON b.book_id = t.book_id
                GROUP BY 1, 2
                HAVING COUNT(t.rating) >= 50) AS ar
            GROUP BY 1, 2
            ORDER BY 3 DESC
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,avg_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844
1,402,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,240,J.R.R. Tolkien,4.258446
3,376,Louisa May Alcott,4.192308
4,498,Rick Riordan,4.080645


Based on the table above, the author with the highest average rating are JK Rowling/Mary GrandPré. Followed closely by Markus Zusak/Cao Xuân Việt Khương and JRR Tolkien.

In [39]:
query = ''' SELECT
                b.title,
                COUNT(t.rating)
            FROM 
                books AS b
                INNER JOIN authors AS a ON b.author_id = a.author_id
                INNER JOIN ratings AS t ON b.book_id = t.book_id
            WHERE a.author_id = 236 OR a.author_id = 402
            GROUP BY 1
        '''

pd.io.sql.read_sql(query, con = engine)

,title,count
0,Harry Potter Boxed Set Books 1-5 (Harry Potte...,2
1,Harry Potter and the Chamber of Secrets (Harry...,80
2,Harry Potter and the Half-Blood Prince (Harry ...,73
3,Harry Potter and the Order of the Phoenix (Har...,75
4,Harry Potter and the Prisoner of Azkaban (Harr...,82
5,The Book Thief,53


### Average number of text reviews from users

In [40]:
query = ''' SELECT
                username,
                COUNT(rating) AS total_rating
            FROM ratings
            GROUP BY 1
            HAVING COUNT(rating) >= 50
        '''

pd.io.sql.read_sql(query, con = engine)

,username,total_rating
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,lesliegibbs,50
5,martinadam,56
6,vanessagardner,50
7,richard89,55
8,shermannatalie,50


There are only 8 users that give more than 50 ratings. We will use this list to find the average number of text reviews from each user.

In [42]:
query = ''' SELECT
                AVG(uv.total_reviews) AS avg_reviews
            FROM
                (SELECT
                ut.username AS username,
                COUNT(v.text) AS total_reviews
                FROM 
                    (SELECT
                        username,
                        COUNT(rating)
                    FROM ratings
                    GROUP BY 1
                    HAVING COUNT(rating) > 50) AS ut
                INNER JOIN reviews AS v ON ut.username = v.username
                GROUP BY 1) AS uv
        '''

pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


Based on the above, we can see that on average users who give more than 50 ratings would give 24 text reviews. 

### Conclusion

We have finished the task and found the following:
- There are 819 books released after January 1, 2000.
- There are 6 books with rating but no text reviews. The rating ranges between 3.0-5.0 
- The average rating is varied depending on the number of reviews. There are some books that have high rating (5.0) but with very little reviews. While books with more than 500 reviews usually rate between 3.5-4.5
- The publisher with most books published is Penguin books with 42 books.
- The author with the highest average book rating is JK Rowling who authored the Harry Potter series. 
- The average number of text reviews from users who give more than 50 ratings is 24 reviews.